# Mexican federal budget pre-processing pipeline

## Instructions

To you run the notebook:

1. choose a unique `ITERATION_LABEL` for each pipeline run
2. specify and describe your input files (`INPUT_FILES`)
3. make sure your column mapping (`COLUMN_ALIASES`) is correct
3. run the whole notebook by clicking on __Kernel > Restart & Run All__

## Imports

In [1]:
from sys import stdout
from pandas import read_csv, concat, DataFrame, ExcelWriter, ExcelFile, Series
from numpy import nan, isnan
from os.path import join, isdir
from os import mkdir
from json import dumps, loads
from pprint import pprint
from csv import DictReader
from shutil import copyfile

## Settings

Choose a unique iteration label for each pipeline run.

In [2]:
ITERATION_LABEL = 'iteration-17-post-launch-bug-fixes-test-run'

Put your input files inside the `pipeline.in` folder and describe them here.

In [3]:
INPUT_FILES = {
    2008: {'name': 'Cuenta_Publica_2008.csv', 'encoding': 'windows-1252'},
    2009: {'name': 'Cuenta_Publica_2009.csv', 'encoding': 'windows-1252'},
    2010: {'name': 'Cuenta_Publica_2010.csv', 'encoding': 'windows-1252'},
    2011: {'name': 'Cuenta_Publica_2011.csv', 'encoding': 'windows-1252'},
    2012: {'name': 'Cuenta_Publica_2012.csv', 'encoding': 'windows-1252'},
    2013: {'name': 'CP_2013.csv', 'encoding': 'windows-1252'},
    2014: {'name': 'CP_2014.csv', 'encoding': 'windows-1252'},
    2015: {'name': 'CP_2015.csv', 'encoding': 'windows-1252'},
    2016: {'name': 'PEF_AC01_2t_2016.csv', 'encoding': 'windows-1252'} 
}


Define your mappings in `columns.aliases.json`.

The following hierarchical categories will have IDs prefixed with the parent categories:

In [4]:
HIERARCHIES = {
    'functional': [
        'GPO_FUNCIONAL', 
        'ID_FUNCION', 
        'ID_SUBFUNCION', 
        'ID_AI'
    ]}

The following columns are unsused and removed at the end of the pipeline:

In [5]:
REMOVE_OUTPUT_COLUMNS = [
    'Reasignacion',
    'Objeto del Gasto',
    'Descripción de Reasignacion',
    'Descripción de Objeto del Gasto'
]

In [6]:
REMOVE_INPUT_COLUMNS = {}

That's it. Now just run the notebook from beginning to end.

## Configuration

In [7]:
BASENAME = 'mexican_federal_budget'
INPUT_FOLDER = 'pipeline.in.v2'
OUTPUT_FOLDER = 'pipeline.out'
ITERATION_FOLDER = join(OUTPUT_FOLDER, ITERATION_LABEL)
MERGED_FILE = join(ITERATION_FOLDER, BASENAME + '.merged.csv')
CATALOGS_FILE = 'objeto_del_gasto.catalog.xlsx'
COLUMN_ALIASES_FILE = 'columns.aliases.json'
DATAPACKAGE_FILE = 'datapackage.default.json'
DESCRIPTIONS_FILE = 'columns.descriptions.csv'

In [8]:
if isdir(ITERATION_FOLDER):
    raise ValueError('Please enter a unique iteration label')
    
mkdir(ITERATION_FOLDER)

## Encoding inspection

Detect the file encodings of the input files using the `cChardet` utility library. __Warning:__ it's not always accurate. This is meant only as an indication only. In the end, encodings will be taken from `INPUT_FILES`.

In [9]:
def detect_encodings():
    """Detect CSV file encoding with the cChardet library"""

    try:
        import cchardet as chardet
    except ImportError:
        cChardet = 'https://github.com/PyYoshi/cChardet'
        print('Encoding inspection skipped: install %s', cChardet)
        return

    results = {}
    results_file = join(OUTPUT_FOLDER, ITERATION_LABEL, 'encodings.detected.json')
    
    for year, file in sorted(INPUT_FILES.items()):
        datafile = join(INPUT_FOLDER, file['name'])
        
        with open(datafile, 'rb') as f:
            text = f.read()
            
        result = chardet.detect(text)
        results.update({year: result})
        print(year, 'Inspected', file['name'], result)
    
    with open(results_file, 'w+') as json:
        json.write(dumps(results, indent=4))
        print('\nSaved encoding detection report to', results_file)
        
# detect_encodings()

## Load files

In [10]:
def read_columns(file, encoding):
    """Return clean CSV file headers"""
    
    with open(file, encoding=encoding) as csv:
        header = csv.readline()
        return header.replace('\n', '').split(',')

In [11]:
def force_strings(columns):
    """Return string enforcement for each column of a CSV file"""
    
    for column in columns:
        yield column, str

In [12]:
def load_csv_files():
    """Load raw data (CSV) files"""
    
    batch = {}
    
    for year, file in sorted(INPUT_FILES.items()):
        filepath = join(INPUT_FOLDER, file['name'])
        column_names = read_columns(filepath, file['encoding'])
        column_types = dict(force_strings(column_names))
        
        batch[year] = read_csv(filepath, encoding=file['encoding'], dtype=column_types)
        print('Loaded', file['name'], 'with encoding', file['encoding'])
    
    print()
    stdout.flush()

    for year in sorted(INPUT_FILES.keys()):
        if year in REMOVE_INPUT_COLUMNS:
            for column in REMOVE_INPUT_COLUMNS[year]:
                try:
                    del batch[year][column]
                    print(year, 'deleted', column)
                except KeyError:
                    print(year, column, 'not found in', file['name'])

        stdout.flush()

    return batch

In [13]:
with open(COLUMN_ALIASES_FILE) as json:
    COLUMN_ALIASES = loads(json.read())
    
with open(DATAPACKAGE_FILE) as json:
    DATAPACKAGE = loads(json.read())

def update_datapackage():
    with open(DESCRIPTIONS_FILE) as stream:
        rows = DictReader(stream)
        reference = {row.pop('Backend Header'): row for row in rows}
   
    fields = DATAPACKAGE['resources'][0]['schema']['fields']
    field_names = []
    
    for field in fields:
        field_name = field['name']
        field['title'] = reference[field_name]['Concept']
        field['Description'] = reference[field_name]['Concept']
        field_names.append(field_name)
        print('updated datapackage title and description for %s' % field_name)
        
    not_matched = set(field_names) ^ set(reference.keys())
    if not_matched:
        raise ValueError('Fields in {} do not match fields in {}: {}'.format(
            DATAPACKAGE_FILE, 
            DESCRIPTIONS_FILE,
            not_matched))
        
    not_matched = set(field_names) ^ set(COLUMN_ALIASES.keys())
    if not_matched:
        raise ValueError('Fields in {} do not match fields in {}: {}'.format(
            DATAPACKAGE_FILE, 
            DESCRIPTIONS_FILE,
            not_matched))
        
    with open(DATAPACKAGE_FILE, 'w+') as stream:
        stream.write(dumps(DATAPACKAGE, indent=2, ensure_ascii=False))
        
    copyfile(DATAPACKAGE_FILE, join(ITERATION_FOLDER, DATAPACKAGE_FILE.replace('.default', '')))
        
    print('checked columns consistency for {}, {} and {}: okay!'.format(
            DATAPACKAGE_FILE, COLUMN_ALIASES_FILE, DESCRIPTIONS_FILE))
    print('copied', DATAPACKAGE_FILE)

## Clean the data

In [14]:
def strip_cell_padding(batch):
    for year in sorted(batch.keys()):
        for column in batch[year].columns:
            batch[year].rename(columns={column: column.strip()}, inplace=True)
            batch[year][column] = batch[year][column].apply(lambda x: x.strip() if x is not nan else x)
        print(year, 'stripped cell paddings')
        stdout.flush()

In [15]:
def delete_empty_columns(batch):
    for year in batch.keys():
        for column in batch[year].columns:
            if 'Unnamed:' in column:
                try:
                    del batch[year][column]
                    print(year, column, 'deleted')
                    stdout.flush()
                except KeyError:
                    pass  

In [16]:
def count_missing_values(batch):
    collector = {}
    table = []

    for column in get_union_of_columns(batch):
        row = {'Column': column}
        collector.update({column: []})
        
        for year in batch.keys():
            if column in batch[year].columns:
                is_empty = batch[year][column].isnull()
                empty_lines = batch[year].where(is_empty).dropna(how='all')
                collector[column].extend(empty_lines.to_dict(orient='records'))
                nb_empty_cells = len(empty_lines)
            else:
                nb_empty_cells = nan
                
            row.update({year: nb_empty_cells})
            if nb_empty_cells not in (nan, 0):
                print(year, 'found', nb_empty_cells, 'missing values in', column)

        table.append(row)
        
    ordered_columns = ['Column']
    ordered_columns.extend(sorted(batch.keys()))
    empty_values_overview_table = DataFrame(table).reindex_axis(ordered_columns, axis=1)
    
    return empty_values_overview_table, collector

In [17]:
def count_duplicates(batch):
    for year, df in sorted(batch.items()):
        nb_duplicate_lines = df.duplicated().apply(lambda x: 1 if x is True else 0).sum()
        print(year, 'found', nb_duplicate_lines, 'duplicate lines')

## Alias column names

In [18]:
def get_union_of_columns(batch):
    union = set()
    for year in batch.keys():
        union = union | set(batch[year].columns)
    return union

In [19]:
from yaml import load

def load_aliases(file):
    with open(file) as yaml:
        aliases = load(yaml.read())
        return aliases

In [20]:
def map_columns_to_aliases(batch, list_of_aliases):
    for year in sorted(batch.keys()):
        for column in sorted(batch[year].columns):
            if not column in list_of_aliases:
                for reference, aliases in list_of_aliases.items():
                    if aliases:
                        if column in aliases:
                            batch[year].rename(columns={column: reference}, inplace=True)
                            print(year, column, 'replaced with', reference)
                            stdout.flush()
                            break  
                else:
                    print(year, 'NO ALIAS REGISTERED FOR', column)
                    stdout.flush()

In [21]:
def build_overview(batch):
    table = []
    
    for column in get_union_of_columns(batch):
        row = {'Column': column}
        for year in batch.keys():
            row.update({year: column in batch[year].columns})
        table.append(row)
        
    ordered_columns = ['Column']
    ordered_columns.extend(sorted(batch.keys()))
    
    overview = DataFrame(table).reindex_axis(ordered_columns, axis=1)
    print('Column mapping overview: done')
    return overview

## Check expenditure sums

There's a little cleaning to do on the amount columns (zeros represented by a dash). Assume thousands are seperated by a comma.

In [22]:
EXPENDITURE_COLUMNS = [
    'MONTO_EJERCIDO', 
    'MONTO_DEVENGADO', 
    'MONTO_APROBADO', 
    'MONTO_PAGADO', 
    'MONTO_MODIFICADO', 
    'MONTO_ADEFAS', 
    'MONTO_EJERCICIO'
]
count = 0

def clean_expenditure_columns(batch):
    check_sums = []

    for column in EXPENDITURE_COLUMNS:
        row = {'Column': column}
        
        for year in sorted(batch.keys()):
            try:
                series = batch[year][column]
                
                # I'm assuming a single '-' represents zero
                series = series.apply(lambda x: '0' if x == '-' else x)
                try:
                    series = series.apply(lambda x: x.replace(',', '') if x is not nan else x)    
                except AttributeError:
                    if count < 10:
                        print(year, column)
                batch[year][column] = series.astype(float)
                check_sum = batch[year][column].sum()
                
                print(year, 'cleaned and summed', column, '=', check_sum, 'pesos')
                
            except KeyError:
                check_sum = nan
                
            row.update({year: check_sum})
        
        check_sums.append(row)

    ordered_columns = ['Column']
    ordered_columns.extend(sorted(batch.keys()))
    return DataFrame(check_sums).reindex_axis(ordered_columns, axis=1)    

## Objeto del Gasto Column split

In [23]:
from os.path import join

def generate_catalog(file):
    
    catalog_ = {}
    catalog_file = ExcelFile(file)
    INDEX_COLUMN = 0
    
    for sheet in catalog_file.sheet_names:
        if sheet != 'Concatenated':
            name = sheet.lower().replace(' ', '_')
            output = join('objeto_del_gasto.catalog', name + '.csv')

            df = catalog_file.parse(sheet).dropna()
            index = df.columns[INDEX_COLUMN]

            df[index] =  df[index].astype(str)
            df.set_index(index, inplace=True)
            df = df.groupby(df.index).first()
            df.sort_index(inplace=True)
            
            message = 'Loaded catalog {sheet} into "{name}" ({nb} lines)'
            parameters = dict(sheet=sheet, name=name, nb=len(df))

            print(message.format(**parameters))
            catalog_[name] = df['DESCRIPCION']
    
    print()
    return catalog_

__Note!__ Years are hard coded in the script below.

In [24]:
def split_objeto_del_gasto(batch):
    catalog = generate_catalog(CATALOGS_FILE)
    missing_in_catalog = []
    
    def has_digits(n, N):
        return not isinstance(n, float) and len(n) >= N 
            
    def lookup(n, table, year):
        try:
            return catalog[table].loc[n]
        except KeyError:
            missing_in_catalog.append({'year': year, 'table': table, 'ID': n})
            return nan
        except TypeError:
            # n is nan
            return nan
    
    for year in sorted(batch.keys()):
        if year == 2016:
            print('Skipping', year, 'because the raw CSV already has the required columns')
        
        else:
            objeto = batch[year]['ID_CONCEPTO'].astype(str)

            batch[year]['ID_CAPITULO'] = objeto.apply(lambda x: x[0] + '000' if x not in (nan, 'nan') else nan)
            batch[year]['ID_CONCEPTO'] = objeto.apply(lambda x: x[:2] + '00' if x not in (nan, 'nan') else nan)
            batch[year]['DESC_CAPITULO'] = batch[year]['ID_CAPITULO'].map(lambda x: lookup(x, 'capitulo', year))  
            batch[year]['DESC_CONCEPTO'] = batch[year]['ID_CONCEPTO'].map(lambda x: lookup(x, 'concepto', year))  
            
            nb_generica_digits = 4 if year in (2008, 2009, 2010) else 3
            
            # Skip the LAST year of the dataset (currently 2016) it has split columns already
            batch[year]['PARTIDA_GENERICA'] = objeto.apply(lambda x: x[:nb_generica_digits] if has_digits(x, 4) else nan)
            batch[year]['DESC_PARTIDA_GENERICA'] = batch[year]['PARTIDA_GENERICA'].map(lambda x: lookup(x, 'partida_generica', year))  
            
            if year not in (2008, 2009, 2010):
                batch[year]['PARTIDA_ESPECIFICA'] = objeto.apply(lambda x: x if has_digits(x, 5) else nan)
                batch[year]['DESC_PARTIDA_ESPECIFICA'] = batch[year]['PARTIDA_ESPECIFICA'].map(lambda x: lookup(x, 'partida_específica', year) if has_digits(x, 5) else nan)  
            else:
                batch[year]['PARTIDA_ESPECIFICA'] = nan
                batch[year]['DESC_PARTIDA_ESPECIFICA'] = nan

            print(year, 'broke down "Objeto del Gasto" column')
        
    return DataFrame(missing_in_catalog).drop_duplicates(['ID', 'table'])

## Prefix IDs 
Disambiguating sub-categories may require prefixing their IDs with their parents' IDs.

In [25]:
def prefix_ids(batch):
    for year in batch.keys():       
        for hierarchy, levels in HIERARCHIES.items():
            prefix = batch[year]['CICLO'].apply(lambda x: '')
            for n, level in enumerate(levels):
                dash = '.' if n > 0 else ''
                prefix = prefix + dash + batch[year][level]  
                batch[year][level] = prefix
                
                print(year, 'prefixed', hierarchy, 'level', n, level)
                stdout.flush()

## Remove unused columns

In [26]:
def remove_unused_columns(batch):
    for year, budget in batch.items():
        for column in REMOVE_OUTPUT_COLUMNS:
            try:
                del budget[column]
                print(year, 'deleted', column)
            except KeyError:
                print(column, ': no such column to delete')

## Add descriptions and titles to datapackage.json

##  Pipeline

In [27]:
def do_pipeline():

    def echo_section(section):
        print('\n', section, '\n')

    echo_section('Update datapackage.json')
    update_datapackage()

    echo_section('Loading files')
    datasets = load_csv_files()
    
    echo_section('Delete empty columns')
    delete_empty_columns(datasets)

    echo_section('Stripping padding from cells')
    strip_cell_padding(datasets)
    
    echo_section('Counting duplicate lines (NOT de-duplicating)')
    count_duplicates(datasets)
    
    echo_section('Mapping column to aliases')
    map_columns_to_aliases(datasets, COLUMN_ALIASES)

    echo_section('Counting missing values')
    missing_values_report, bad_records = count_missing_values(datasets)
    
    echo_section('Building column mapping overview')
    column_mapping_report = build_overview(datasets)
    
    echo_section('Cleaning expenditure columns')
    sums_report = clean_expenditure_columns(datasets)
    
    echo_section('Breaking down Objeto del Gasto column')
    missing_catalog_ids = split_objeto_del_gasto(datasets)
        
    echo_section('Prefixing IDs of certain category hierarchies')
    prefix_ids(datasets)

    echo_section('Removing unused columns')
    remove_unused_columns(datasets)

    echo_section('Saving pipeline configuration')

    reports_file = join(ITERATION_FOLDER, BASENAME + '.reports.xlsx')
    writer = ExcelWriter(reports_file)    
    missing_values_report.to_excel(writer, 'missing values', encoding='utf-8', index=False)
    column_mapping_report.to_excel(writer, 'column mapping', encoding='utf-8', index=False)
    sums_report.to_excel(writer, 'check sums', encoding='utf-8', index=False)
    missing_catalog_ids.to_excel(writer, 'missing_catalog_IDs', encoding='utf-8', index=False)    
    print('Saved 4 reports to', reports_file)    

    aliases_file = join(ITERATION_FOLDER, BASENAME + '.aliases.json')
    inputs_file = join(ITERATION_FOLDER, BASENAME + '.inputs.json')
    levels_file = join(ITERATION_FOLDER, BASENAME + '.levels.json')
    bad_records_file = join(ITERATION_FOLDER, BASENAME + '.missing.json')

    with open(bad_records_file, 'w+') as json:
        json.write(dumps(bad_records, indent=4))
        
    with open(aliases_file, 'w+') as json:
        json.write(dumps(COLUMN_ALIASES, indent=4))
        
    with open(levels_file, 'w+') as json:
        json.write(dumps(HIERARCHIES, indent=4))
        
    with open(inputs_file, 'w+') as json:
        json.write(dumps(INPUT_FILES, indent=4))
    
    print('Saved input configuration to', inputs_file)    
    print('Saved column aliases to', aliases_file) 
    print('Saved bad records (those with empty cells) to', bad_records_file)    
    print('Saved hierarchy levels used for prefixing to', levels_file) 
    
    echo_section('Pipeline run "%s" done' % ITERATION_LABEL)

    return datasets, missing_catalog_ids, column_mapping_report, missing_values_report, sums_report

## Run the pipeline

In [28]:
budgets, missing_ids, column_mapping, missing_values, sums = do_pipeline()


 Update datapackage.json 

updated datapackage title and description for CICLO
updated datapackage title and description for DESC_AI
updated datapackage title and description for DESC_CAPITULO
updated datapackage title and description for DESC_CONCEPTO
updated datapackage title and description for ENTIDAD_FEDERATIVA
updated datapackage title and description for DESC_FF
updated datapackage title and description for DESC_FUNCION
updated datapackage title and description for DESC_GPO_FUNCIONAL
updated datapackage title and description for DESC_MODALIDAD
updated datapackage title and description for DESC_PARTIDA_ESPECIFICA
updated datapackage title and description for DESC_PARTIDA_GENERICA
updated datapackage title and description for DESC_PP
updated datapackage title and description for DESC_RAMO
updated datapackage title and description for DESC_SUBFUNCION
updated datapackage title and description for DESC_TIPOGASTO
updated datapackage title and description for DESC_UR
updated datapacka

In [29]:
from gc import collect
collect()

3045

In [30]:
for year, budget in budgets.items():
    filepath = MERGED_FILE.replace('merged', str(year))
    budget.to_csv(filepath, encoding='utf-8', index=False)
    print('Saved', filepath)
    stdout.flush()

Saved pipeline.out/iteration-17-post-launch-bug-fixes-test-run/mexican_federal_budget.2016.csv
Saved pipeline.out/iteration-17-post-launch-bug-fixes-test-run/mexican_federal_budget.2008.csv
Saved pipeline.out/iteration-17-post-launch-bug-fixes-test-run/mexican_federal_budget.2009.csv
Saved pipeline.out/iteration-17-post-launch-bug-fixes-test-run/mexican_federal_budget.2010.csv
Saved pipeline.out/iteration-17-post-launch-bug-fixes-test-run/mexican_federal_budget.2011.csv
Saved pipeline.out/iteration-17-post-launch-bug-fixes-test-run/mexican_federal_budget.2012.csv
Saved pipeline.out/iteration-17-post-launch-bug-fixes-test-run/mexican_federal_budget.2013.csv
Saved pipeline.out/iteration-17-post-launch-bug-fixes-test-run/mexican_federal_budget.2014.csv
Saved pipeline.out/iteration-17-post-launch-bug-fixes-test-run/mexican_federal_budget.2015.csv


In [31]:
merged = concat(list(budgets.values()))
merged.to_csv(MERGED_FILE, encoding='utf-8', index=False)
print('Saved merged dataset to', MERGED_FILE)    

Saved merged dataset to pipeline.out/iteration-17-post-launch-bug-fixes-test-run/mexican_federal_budget.merged.csv


## Quality control

In [32]:
sorted(list(budget.columns))

['CICLO',
 'DESC_AI',
 'DESC_CAPITULO',
 'DESC_CONCEPTO',
 'DESC_FF',
 'DESC_FUNCION',
 'DESC_GPO_FUNCIONAL',
 'DESC_MODALIDAD',
 'DESC_PARTIDA_ESPECIFICA',
 'DESC_PARTIDA_GENERICA',
 'DESC_PP',
 'DESC_RAMO',
 'DESC_SUBFUNCION',
 'DESC_TIPOGASTO',
 'DESC_UR',
 'ENTIDAD_FEDERATIVA',
 'GPO_FUNCIONAL',
 'ID_AI',
 'ID_CAPITULO',
 'ID_CLAVE_CARTERA',
 'ID_CONCEPTO',
 'ID_ENTIDAD_FEDERATIVA',
 'ID_FF',
 'ID_FUNCION',
 'ID_MODALIDAD',
 'ID_PP',
 'ID_RAMO',
 'ID_SUBFUNCION',
 'ID_TIPOGASTO',
 'ID_UR',
 'MONTO_ADEFAS',
 'MONTO_APROBADO',
 'MONTO_DEVENGADO',
 'MONTO_EJERCICIO',
 'MONTO_MODIFICADO',
 'MONTO_PAGADO',
 'PARTIDA_ESPECIFICA',
 'PARTIDA_GENERICA']

In [33]:
merged.sample(n=20)

,CICLO,DESC_AI,DESC_CAPITULO,DESC_CONCEPTO,DESC_FF,DESC_FUNCION,DESC_GPO_FUNCIONAL,DESC_MODALIDAD,DESC_PARTIDA_ESPECIFICA,DESC_PARTIDA_GENERICA,...,ID_UR,MONTO_ADEFAS,MONTO_APROBADO,MONTO_DEVENGADO,MONTO_EJERCICIO,MONTO_EJERCIDO,MONTO_MODIFICADO,MONTO_PAGADO,PARTIDA_ESPECIFICA,PARTIDA_GENERICA
268895,2012,Sistema Nacional de Información Estadística y ...,Materiales y suministros,"Materiales de administracion, emision de docum...",Recursos fiscales,Otros Servicios Generales,Gobierno,"Planeación, seguimiento y evaluación de políti...",Materiales y útiles de oficina,"Materiales, útiles y equipos menores de oficina",...,100,NaN,27606.0,NaN,NaN,0.00,NaN,NaN,21101,211
220038,2014,Impartición de Justicia en materia agraria,Servicios generales,Servicios basicos,Recursos fiscales,Justicia,Gobierno,Prestación de Servicios Públicos,Servicio telegráfico,Servicios postales y telegráficos,...,200,0.0,0.0,0.00,0.00,NaN,0.00,0.00,31802,318
299538,2013,Prestaciones sociales y culturales,Servicios generales,"Servicios de instalacion, reparacion, mantenim...",Ingresos Propios,Protección Social,Desarrollo Social,Prestación de Servicios Públicos,Mantenimiento y conservación de maquinaria y e...,"Instalación, reparación y mantenimiento de maq...",...,GYN,NaN,0.0,NaN,92296.00,NaN,NaN,NaN,35701,357
317379,2014,Cobertura de la atención médica curativa,Servicios personales,Otras prestaciones sociales y económicas,Ingresos Propios,Salud,Desarrollo Social,Prestación de Servicios Públicos,Cuotas para el fondo de ahorro del personal civil,Cuotas para el fondo de ahorro y fondo de trabajo,...,GYN,0.0,36781.0,0.00,0.00,NaN,0.00,0.00,15101,151
94970,2009,Instrumentación de la política Laboral,Materiales y suministros,Alimentos y utensilios,Recursos fiscales,Temas Laborales,Desarrollo Económico,Prestación de Servicios Públicos,NaN,Productos alimenticios,...,153,NaN,0.0,NaN,NaN,0.00,NaN,NaN,NaN,2200
286456,2015,Cobertura de la atención médica curativa,Servicios personales,Remuneraciones adicionales y especiales,Ingresos Propios,Salud,Desarrollo Social,Prestación de Servicios Públicos,Prima quinquenal por años de servicios efectiv...,Primas por años de servicios efectivos prestados,...,GYN,0.0,0.0,2215.00,2215.00,NaN,2215.00,2215.00,13101,131
61871,2013,"Carreteras eficientes, seguras y suficientes",Inversion publica,Obra publica en bienes propios,Recursos fiscales,Transporte,Desarrollo Económico,Proyectos de Inversión,Servicios de supervisión de obras,Trabajos de acabados en edificaciones y otros ...,...,639,NaN,3000000.0,NaN,2399279.38,NaN,NaN,NaN,62903,629
99668,2013,Educación superior de calidad,Servicios personales,Seguridad social,Recursos fiscales,Educación,Desarrollo Social,"Planeación, seguimiento y evaluación de políti...",Cuotas para el seguro de vida del personal civil,Aportaciones para seguros,...,500,NaN,188806.0,NaN,208882.87,NaN,NaN,NaN,14401,144
7738,2008,Función pública y buen gobierno,Materiales y suministros,"Materiales de administracion, emision de docum...",Recursos fiscales,Administración Pública,Gobierno,Actividades de apoyo a la función pública y al...,NaN,Materiales y útiles de oficina,...,113,NaN,86607.0,NaN,NaN,59959.00,NaN,NaN,NaN,2101
128492,2009,Democracia preservada y fortalecida mediante l...,Servicios generales,"Servicios de instalacion, reparacion, mantenim...",Recursos fiscales,Gobernación,Gobierno,Apoyo al proceso presupuestario y para mejorar...,NaN,Mantenimiento y conservación de mobiliario y e...,...,200,NaN,1027889.0,NaN,NaN,846496.00,NaN,NaN,NaN,3501


In [34]:
objeto_breakdown = [
    'CICLO', 
    'ID_CAPITULO', 
    'ID_CONCEPTO', 
    'PARTIDA_ESPECIFICA', 
    'PARTIDA_GENERICA'
]
merged[objeto_breakdown].sample(n=20)

,CICLO,ID_CAPITULO,ID_CONCEPTO,PARTIDA_ESPECIFICA,PARTIDA_GENERICA
151830,2014,6000,6200,62903,629
209336,2015,1000,1300,13101,131
22085,2010,3000,3600,NaN,3602
65065,2009,3000,3500,NaN,3505
316554,2014,1000,1400,14101,141
200240,2015,3000,3300,33605,336
106675,2010,3000,3800,NaN,3808
65810,2009,1000,1300,NaN,1301
864,2008,3000,3100,NaN,3100
45009,2015,3000,3100,31401,314


In [35]:
print('Total: missing', len(missing_ids), 'catalog IDs to breakdown the "Objeto del Gasto" column')
print('Tables:', dict(missing_ids.groupby('table').count()['ID']))
print('Years:', dict(missing_ids.groupby('year').count()['ID']))
try:
    missing_ids.sample(n=20)
except ValueError:
    pass

Total: missing 76 catalog IDs to breakdown the "Objeto del Gasto" column
Tables: {'partida_específica': 24, 'partida_generica': 45, 'concepto': 7}
Years: {2008: 48, 2009: 4, 2012: 22, 2013: 1, 2015: 1}


In [36]:
missing_ids.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76 entries, 0 to 18706
Data columns (total 3 columns):
ID       76 non-null object
table    76 non-null object
year     76 non-null int64
dtypes: int64(1), object(2)
memory usage: 2.4+ KB


In [37]:
column_mapping

,Column,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,DESC_AI,True,True,True,True,True,True,True,True,True
1,DESC_PP,True,True,True,True,True,True,True,True,True
2,MONTO_EJERCICIO,False,False,False,False,False,True,True,True,False
3,ID_PP,True,True,True,True,True,True,True,True,True
4,ID_CLAVE_CARTERA,False,False,False,False,True,True,True,True,True
5,DESC_TIPOGASTO,True,True,True,True,True,True,True,True,True
6,MONTO_MODIFICADO,False,False,False,False,False,False,True,True,False
7,ID_RAMO,True,True,True,True,True,True,True,True,True
8,MONTO_DEVENGADO,False,False,False,False,False,False,True,True,False
9,DESC_SUBFUNCION,True,True,True,True,True,True,True,True,True


In [38]:
missing_values

,Column,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,DESC_AI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,DESC_PP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,MONTO_EJERCICIO,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN
3,ID_PP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ID_CLAVE_CARTERA,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
5,DESC_TIPOGASTO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,MONTO_MODIFICADO,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
7,ID_RAMO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,MONTO_DEVENGADO,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
9,DESC_SUBFUNCION,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
sums

,Column,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,MONTO_EJERCIDO,2.576692e+12,2.296086e+12,2.474100e+12,2.717372e+12,2.896331e+12,NaN,NaN,NaN,2.160939e+12
1,MONTO_DEVENGADO,NaN,NaN,NaN,NaN,NaN,NaN,5.076810e+12,5.508987e+12,NaN
2,MONTO_APROBADO,1.992356e+12,2.289715e+12,2.376915e+12,2.560196e+12,2.754868e+12,4.322619e+12,4.905401e+12,5.138442e+12,5.297126e+12
3,MONTO_PAGADO,NaN,NaN,NaN,NaN,NaN,NaN,4.972712e+12,5.366652e+12,2.137679e+12
4,MONTO_MODIFICADO,NaN,NaN,NaN,NaN,NaN,NaN,5.013990e+12,5.398608e+12,NaN
5,MONTO_ADEFAS,NaN,NaN,NaN,NaN,NaN,NaN,3.694161e+10,3.112265e+10,NaN
6,MONTO_EJERCICIO,NaN,NaN,NaN,NaN,NaN,4.617618e+12,5.010877e+12,5.399018e+12,NaN


In [40]:
merged.sample(n=20)

,CICLO,DESC_AI,DESC_CAPITULO,DESC_CONCEPTO,DESC_FF,DESC_FUNCION,DESC_GPO_FUNCIONAL,DESC_MODALIDAD,DESC_PARTIDA_ESPECIFICA,DESC_PARTIDA_GENERICA,...,ID_UR,MONTO_ADEFAS,MONTO_APROBADO,MONTO_DEVENGADO,MONTO_EJERCICIO,MONTO_EJERCIDO,MONTO_MODIFICADO,MONTO_PAGADO,PARTIDA_ESPECIFICA,PARTIDA_GENERICA
218436,2012,Democracia preservada y fortalecida mediante l...,Servicios generales,"Servicios profesionales, cientificos, tecnicos...",Recursos fiscales,Coordinación de la Política de Gobierno,Gobierno,Específicos,Otras asesorías para la operación de programas,"Servicios legales, de contabilidad, auditoría ...",...,115,NaN,1.605001e+07,NaN,NaN,0.000000e+00,NaN,NaN,33104,331
276541,2015,Cobertura de la atención médica preventiva,Servicios personales,Remuneraciones adicionales y especiales,Ingresos Propios,Salud,Desarrollo Social,Prestación de Servicios Públicos,Compensaciones adicionales por servicios espec...,Compensaciones,...,GYN,0.0,0.000000e+00,38778.00,38778.00,NaN,38778.00,38778.00,13407,134
177162,2013,Incremento de la oferta turística orientada a ...,Inversion publica,Obra publica en bienes propios,Recursos fiscales,Turismo,Desarrollo Económico,Proyectos de Inversión,Servicios de supervisión de obras,Trabajos de acabados en edificaciones y otros ...,...,W3N,NaN,1.071565e+06,NaN,1971396.64,NaN,NaN,NaN,62903,629
337332,2014,Cobertura de la atención médica preventiva,Servicios personales,Remuneraciones adicionales y especiales,Recursos fiscales,Salud,Desarrollo Social,Prestación de Servicios Públicos,Prima quinquenal por años de servicios efectiv...,Primas por años de servicios efectivos prestados,...,GYN,0.0,4.765000e+03,4401.00,4401.00,NaN,4401.00,4401.00,13101,131
175,2009,Fiscalización de la Cuenta de la Hacienda Públ...,Materiales y suministros,Materias primas y materiales de produccion y c...,Recursos fiscales,Legislación,Gobierno,Específicos,NaN,Utensilios para el servicio de alimentación,...,101,NaN,3.885000e+03,NaN,NaN,1.315600e+04,NaN,NaN,NaN,2303
23836,2015,Servicios de tesorería eficientes y transparentes,Servicios generales,"Servicios profesionales, cientificos, tecnicos...",Recursos fiscales,Asuntos Financieros y Hacendarios,Gobierno,Prestación de Servicios Públicos,Servicios de vigilancia,Servicios de vigilancia,...,613,18455.0,1.731540e+05,227413.00,227413.00,NaN,227413.00,208958.00,33801,338
254232,2013,"Oportunidad en la atención curativa, quirúrgic...",Servicios personales,Otras prestaciones sociales y económicas,Recursos fiscales,Salud,Desarrollo Social,Apoyo al proceso presupuestario y para mejorar...,Prestaciones establecidas por condiciones gene...,Prestaciones contractuales,...,GYR,NaN,3.078660e+05,NaN,305305.20,NaN,NaN,NaN,15401,154
33993,2012,Tecnificación e innovación de las actividades ...,Materiales y suministros,"Herramientas, refacciones y accesorios menores",Recursos fiscales,"Ciencia, Tecnología e Innovación",Desarrollo Económico,Prestación de Servicios Públicos,Refacciones y accesorios menores de mobiliario...,Refacciones y accesorios menores de mobiliario...,...,H00,NaN,0.000000e+00,NaN,NaN,1.878000e+03,NaN,NaN,29301,293
150584,2012,Ordenamiento y regularización de la propiedad ...,Servicios personales,Remuneraciones adicionales y especiales,Recursos fiscales,"Agropecuaria, Silvicultura, Pesca y Caza",Desarrollo Económico,"Planeación, seguimiento y evaluación de políti...",Primas de vacaciones y dominical,"Primas de vacaciones, dominical y gratificació...",...,133,NaN,8.705300e+04,NaN,NaN,8.590630e+04,NaN,NaN,13201,132
46485,2014,Regulación eficiente de las comunicaciones y l...,Servicios generales,Servicios de traslado y viáticos,Recursos fiscales,Transporte,Desarrollo Económico,Regulación y supervisión,Viáticos nacionales para servidores públicos e...,Viáticos en el país,...,623,0.0,2.033600e+05,102137.20,102137.20,NaN,102137.20,102137.20,37504,375


In [41]:
breakdown = [
    'CICLO', 
    'ID_CAPITULO', 
    'ID_CONCEPTO', 
    'PARTIDA_GENERICA',        
    'PARTIDA_ESPECIFICA', 
    'DESC_CAPITULO',
    'DESC_CONCEPTO', 
    'DESC_PARTIDA_GENERICA',
    'DESC_PARTIDA_ESPECIFICA'
]

merged[breakdown].sample(n=200)

,CICLO,ID_CAPITULO,ID_CONCEPTO,PARTIDA_GENERICA,PARTIDA_ESPECIFICA,DESC_CAPITULO,DESC_CONCEPTO,DESC_PARTIDA_GENERICA,DESC_PARTIDA_ESPECIFICA
72651,2016,1000,1500,NaN,NaN,Servicios personales,Otras prestaciones sociales y económicas,NaN,NaN
258964,2015,1000,1500,151,15101,Servicios personales,Otras prestaciones sociales y económicas,Cuotas para el fondo de ahorro y fondo de trabajo,Cuotas para el fondo de ahorro del personal civil
12846,2015,1000,1600,161,16104,Servicios personales,Previsiones,"Previsiones de carácter laboral, económica y d...",Previsiones para aportaciones al ISSSTE
35734,2014,3000,3200,323,32301,Servicios generales,Servicios de arrendamiento,Arrendamiento de mobiliario y equipo de admini...,Arrendamiento de equipo y bienes informáticos
254186,2015,3000,3800,383,38301,Servicios generales,Servicios oficiales,Congresos y convenciones,Congresos y convenciones
48935,2015,3000,3300,336,33602,Servicios generales,"Servicios profesionales, cientificos, tecnicos...","Servicios de apoyo administrativo, traducción,...",Otros servicios comerciales
115218,2013,1000,1500,154,15401,Servicios personales,Otras prestaciones sociales y económicas,Prestaciones contractuales,Prestaciones establecidas por condiciones gene...
165034,2014,3000,3500,353,35301,Servicios generales,"Servicios de instalacion, reparacion, mantenim...","Instalación, reparación y mantenimiento de equ...",Mantenimiento y conservación de bienes informá...
272946,2014,2000,2100,216,21601,Materiales y suministros,"Materiales de administracion, emision de docum...",Material de limpieza,Material de limpieza
295291,2015,1000,1500,159,15901,Servicios personales,Otras prestaciones sociales y económicas,Otras prestaciones sociales y económicas,Otras prestaciones


In [42]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1909649 entries, 0 to 341255
Data columns (total 39 columns):
CICLO                      object
DESC_AI                    object
DESC_CAPITULO              object
DESC_CONCEPTO              object
DESC_FF                    object
DESC_FUNCION               object
DESC_GPO_FUNCIONAL         object
DESC_MODALIDAD             object
DESC_PARTIDA_ESPECIFICA    object
DESC_PARTIDA_GENERICA      object
DESC_PP                    object
DESC_RAMO                  object
DESC_SUBFUNCION            object
DESC_TIPOGASTO             object
DESC_UR                    object
ENTIDAD_FEDERATIVA         object
GPO_FUNCIONAL              object
ID_AI                      object
ID_CAPITULO                object
ID_CLAVE_CARTERA           object
ID_CONCEPTO                object
ID_ENTIDAD_FEDERATIVA      object
ID_FF                      object
ID_FUNCION                 object
ID_MODALIDAD               object
ID_PP                      object
ID_R

In [43]:
len(merged)

1909649

In [44]:
merged.sample(n=10000).to_csv('pipeline.out/iteration-15-all-new-data-2008-2016/mexican_federal_budget_sample.csv')

In [45]:
merged['CICLO'].value_counts()

2015    341256
2014    340699
2013    308453
2012    272142
2009    146382
2011    142549
2008    130950
2010    129014
2016     98204
Name: CICLO, dtype: int64